<h2><b>Automate & document iMOD runs</b></h2>

This script works as a wrapper for the iMOD runs performed for the WaterScape project. Inputs are read from the RunLog, where the runs are documented. Then those inputs are used to run the model and PoP the output, in an automated and documented way.<br>
This is also necessary because additional functions, from WS_Mdl.py are run in between iMOD functions. Hence a "coordinator" is needed.

# 0. Prep

## 0.0. Libraries

In [1]:
import WS_Mdl as WS
import shutil as sh
import os
import openpyxl as xl
from openpyxl.styles import PatternFill
import pandas as pd
from datetime import datetime as DT, timezone as TZ, timedelta
import subprocess as sp
from multiprocessing import Process, cpu_count

## 0.1. Paths

In [2]:
path_RunLog = r"../Mng/WS_RunLog.xlsx"
# get_MdlN_paths()

# 1. Load RunLog and read inputs

In [3]:
TZ_UTC1 = TZ(timedelta(hours=1))  # Define UTC+1 timezone
DT_Script_Start = DT.now(TZ_UTC1)
print(f"-- RunMng initiated at {DT_Script_Start.strftime('%Y-%m-%d %H:%M:%S.%f')[:23]}")

-- RunMng initiated at 2025-03-27 10:28:42.016


## 1.0. Read RunLog with openpyxl
Allows value modifications without overwritting formatting

In [4]:
sh.copy2(path_RunLog, f"../Mng/SS/WS_RunLog_{DT_Script_Start.strftime(format="%Y%m%d_%H%M%S")}.xlsx") # Make a copy of RunLog just in case.

'../Mng/SS/WS_RunLog_20250327_102842.xlsx'

In [5]:
# WB = xl.load_workbook(path_RunLog, data_only=True) # Load the workbook #666 unecessary as we can do the same with pandas, but much faster.
WB = xl.load_workbook(path_RunLog)['RunLog'] # Load the workbook - write. Only RunLog tab will be written to.

In [6]:
# WB_RL, WB_PP_Ctrl, WB_PP_Opt = WB['RunLog'], WB['PP_Ctrl'], WB['PP_Opt']

In [7]:
# N_Ln = -1
# for R in WB.iter_rows(values_only=True, max_row=20):
#     print(R)
#     if not ( (isinstance(R[0], int)) or (R[0]=='runN') ):
#         break
#     N_Ln += 1
# N_Ln

## 1.1 Read with pd for easier work

In [21]:
DF = pd.read_excel(path_RunLog, sheet_name='RunLog').dropna(subset='runN')
DF_Opt = pd.read_excel(path_RunLog, skiprows=1, sheet_name='PP_Opt').dropna(subset='MdlN')

In [23]:
DF_q = DF.loc[DF['Status'] == 'Queued'] # _q for queued

In [ ]:
if __name__ == "__main__":

    num_cores = min(len(DF_q), cpu_count())  # Determine max number of cores to use
    processes = []  # Store processes

    for i, Se_Ln in DF_q.iterrows():  # Start a separate process for each model
        p = Process(target=WS.run_Mdl, args=(Se_Ln, DF_Opt))
        processes.append(p)
        p.start()

    for p in processes:  # Ensure all processes complete
        p.join()

In [ ]:
stop

# Junkyard

In [ ]:
stop

In [ ]:
d_Cmds.items()

In [ ]:
# Run models in parallel using a process pool
with Pool(num_cores) as pool:
    pool.starmap(WS.run_Mdl, [(row, DF_Opt) for _, row in queued_Sims.iterrows()])

In [ ]:
for i, Se_Ln in DF.loc[DF['Status']=='Queued'].iterrows():
    MdlN = Se_Ln['MdlN'] # Get MdlN from Se
    print(f"--- Executing run_Mdl for {MdlN}")

    d_paths = WS.get_MdlN_paths(MdlN) # Get default directories
    MdlN_B, path_Mdl, path_MdlN, path_INI, path_BAT, path_PRJ = (d_paths[k] for k in ['MdlN_B','path_Mdl', 'path_MdlN', "path_INI_S", "path_BAT_S", "path_PRJ_S"]) # and pass them to objects that will be used in the function #666 some of those may be unecessary. Check and remove before conveting to .py
        
    d_Cmds = {path_BAT: os.path.dirname(path_BAT),
            "activate imod": os.path.dirname(path_BAT),
            f"WS_Mdl add_OBS {MdlN} {DF_Opt.loc[DF_Opt['MdlN']==MdlN, 'add_OBS'].values[0]}": os.path.dirname(path_BAT),
            'RUN.BAT': path_MdlN} # contains command and cwd path for command to be run in.

    with open(os.path.join(path_MdlN, f'Tmnl_Out_{MdlN}.txt'), 'w') as f:
        for Cmd in d_Cmds:
            try:
                print(f" -- Executing: {Cmd}")
                Tmnl_Out = sp.run(Cmd, shell=True, capture_output=True, text=True, check=True, cwd=d_Cmds[Cmd])
                print(f"  - \u2713")

                f.write(f'{"*"*50}  START OF COMMAND   {"*"*50}\n')
                f.write(f"Command: {Tmnl_Out.args}\n{"-"*120}\n\n")
                f.write(f"Output:\n{Tmnl_Out.stdout}\n{"-"*120}\n\n")
                f.write(f"Errors:\n{Tmnl_Out.stderr}\n{"-"*120}\n\n")
                f.write(f"Return Code: {Tmnl_Out.returncode}\n{"-"*120}\n\n")
                f.write(f'{"*"*50}  END OF COMMAND     {"*"*50}\n\n')
            except sp.CalledProcessError as e:
                print(f"  - \u274C: {Cmd}\nError: {e.stderr}")

In [ ]:
s = 'NBr1'
Mdl, SimN = "".join(filter(str.isalpha, s)), "".join(filter(str.isdigit, s))
Mdl, SimN

## Examples

In [ ]:
# Change the color of a specific cell (e.g., A1)
fill = PatternFill(start_color="FFFF00", end_color="FFFF00", fill_type="solid")  # Yellow color
sheet['A1'].fill = fill

In [ ]:
sheet['F5'] = 'Development'
sheet['F6'] = 'development'
sheet['F7'] = 'Development_'

In [ ]:
# Save the modified workbook
wb.save('modified_file.xlsx')